## Week 3 Final Assignment Part 3 (Continuation from Part 1 & Part 2) - Segmenting and Clustering Neighbourhoods in Toronto, ON.

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe as per example in the project instructions.

Detailed steps below


Step 1: 3 COLUMNS ARE CREATED FOR THIS DATAFRAME : PostalCode, Borough and Neighborhood

In [1]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

# I did not use Beatiful Soup for this as I felt this would be easier and faster
required_cols = ['Postal Code', 'Borough','Neighbourhood']

# Note I converted the boolean output of array_equal function to String as it 
# wasn't working as expected when used in the if condition as expected
for table in tables:
    if(str(np.array_equal(np.array(table.columns),np.array(required_cols)))=="True"):
        pstl_data_df = pd.DataFrame(table)    
    break
print("Shape of Dataframe is - ", pstl_data_df.shape)
pstl_data_df.head()

Shape of Dataframe is -  (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Step 2: Process only the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
#Filtering out Boroughs which are Not assigned
pstl_data_df = pstl_data_df[pstl_data_df.Borough!="Not assigned"]
print("Shape of Dataframe is - ",pstl_data_df.shape)
pstl_data_df.head()

Shape of Dataframe is -  (103, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Step 3: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [4]:
pstl_data_df['new_nghbr'] = np.where(pstl_data_df['Neighbourhood']=='Not assigned',pstl_data_df['Borough'],pstl_data_df['Neighbourhood'])
pstl_data_df.head(11)

,Postal Code,Borough,Neighbourhood,new_nghbr
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village","Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge","Malvern, Rouge"
11,M3B,North York,Don Mills,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens","Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson","Garden District, Ryerson"


Step 4: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

Used the "new_nghbr" column to get a grouped column, as in wherever the same Postal Code and the same Borough are present we have to club the Neighbourhood column using the functions groupby and apply

In [5]:
can_postal_cd_df = pd.DataFrame(pstl_data_df.groupby(['Postal Code','Borough'])['new_nghbr'].apply(','.join).reset_index())

# Renaming Columns to the same one in the image given:
can_postal_cd_df = can_postal_cd_df.rename(columns = {"Postal Code": "Postal Code","new_nghbr":"Neighborhood"})

In [6]:
print(can_postal_cd_df.head())
print("\n The Final Shape of the dataframe is  - ",can_postal_cd_df.shape)

  Postal Code      Borough                            Neighborhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae

 The Final Shape of the dataframe is  -  (103, 3)


As We see above the shape of the new data frame created is (103,3)

## PART 1 ENDS HERE

## PART 2 ADDING THE LAT LONG DATA TO THE EXISTING DATAFRAME

Step 1: Installing Geocoder

In [7]:
!pip install geocoder

In [8]:
import geocoder # import geocoder
print("Geo Coder imported!")

Geo Coder imported!



Creating a function to get the Lat Long data from the Postal Code

In [9]:
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

Adding the Latitude and Longitude columns to the Pandas DataFrame

In [10]:
can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'] = zip(*can_postal_cd_df['Postal Code'].apply(get_geocoder))
can_postal_cd_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [11]:
print("Shape of the dataframe is - ",can_postal_cd_df.shape)

Shape of the dataframe is -  (103, 5)


## Map of Toronto

In [12]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Ontario are 43.6534817, -79.3839347.


Using Folium to Map

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'], can_postal_cd_df['Postal Code'], can_postal_cd_df['Borough'], can_postal_cd_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## PART-2 Ends Here


## PART-3 Explore and Cluster Neighbourhoods in Toronto

# Step - 1: Filtering out Toronto Boroughs in the complete Data

In [14]:
# Printing all Borough values in the Pandas Dataframe
can_postal_cd_df.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [15]:
toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_all_df = can_postal_cd_df[can_postal_cd_df['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print("Shape of All Toronto Data dataframe is - ",toronto_all_df.shape)
toronto_all_df.head(40)

Shape of All Toronto Data dataframe is -  (39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.67709,-79.29547
1,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.66797,-79.31467
3,M4M,East Toronto,Studio District,43.66213,-79.33497
4,M4N,Central Toronto,Lawrence Park,43.72843,-79.38713
5,M4P,Central Toronto,Davisville North,43.71276,-79.38851
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.71458,-79.40668
7,M4S,Central Toronto,Davisville,43.70340,-79.38659
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.69048,-79.38318
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.68568,-79.40237


In [17]:
#Mapping only Toronto Data which was filtered earlier, we can see the variance in both maps which had the locations marked
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, post, borough, neigh in zip(toronto_all_df['Latitude'], toronto_all_df['Longitude'], toronto_all_df['Postal Code'], toronto_all_df['Borough'], toronto_all_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

### Step - 2:Using Foursquare API going forward:

In [30]:
CLIENT_ID = 'GUCQFRNNVC235LWP2PZE3H2MAX5Q4Q1EUULGPXEBD22ABAM1' # your Foursquare ID
CLIENT_SECRET = 'UKJ0ACTS3R3AI0R1YNNHELHFV1CL4ZKTWWVWMHYEFKN0SXTA' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GUCQFRNNVC235LWP2PZE3H2MAX5Q4Q1EUULGPXEBD22ABAM1
CLIENT_SECRET:UKJ0ACTS3R3AI0R1YNNHELHFV1CL4ZKTWWVWMHYEFKN0SXTA


#### All recommended places in each Borough in Toronto

In [31]:
!pip install requests

In [32]:
import requests
radius = 500
LIMIT = 1000

recommends = []
for lat, long, post, borough, neighborhood in zip(toronto_all_df['Latitude'], toronto_all_df['Longitude'], toronto_all_df['Postal Code'], toronto_all_df['Borough'], toronto_all_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    postal_data = requests.get(url).json()["response"]['groups'][0]['items']
    for recommend_post in postal_data:
        recommends.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            recommend_post['venue']['name'], 
            recommend_post['venue']['location']['lat'], 
            recommend_post['venue']['location']['lng'],  
            recommend_post['venue']['categories'][0]['name']))

In [33]:
toronto_recommends = pd.DataFrame(recommends)
toronto_recommends.columns = ['Postal Code', 'Borough', 'Neighborhoods', 'Borough_Lat', 'Borough_Long', 'Venue_Name', 'Venue_Lat', 'Venue_Long', 'Venue_Category']
print("Shape of all the recommendations of all the Postal Codes in Toronto - ", toronto_recommends.shape)
toronto_recommends.head()

Shape of all the recommendations of all the Postal Codes in Toronto -  (1752, 9)


,Postal Code,Borough,Neighborhoods,Borough_Lat,Borough_Long,Venue_Name,Venue_Lat,Venue_Long,Venue_Category
0,M4E,East Toronto,The Beaches,43.67709,-79.29547,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.67709,-79.29547,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.67709,-79.29547,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.67709,-79.29547,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512,Dairy Queen,43.684223,-79.357062,Ice Cream Shop


In [34]:
#Removing any duplicates if any
toronto_recommends.drop_duplicates(keep=False, inplace=True)
toronto_recommends.shape

(1752, 9)

In [36]:
toronto_recommends.groupby(['Postal Code', 'Borough', 'Neighborhoods'])['Venue_Name'].count()

Postal Code  Borough           Neighborhoods                                                                                             
M4E          East Toronto      The Beaches                                                                                                     4
M4K          East Toronto      The Danforth West, Riverdale                                                                                    9
M4L          East Toronto      India Bazaar, The Beaches West                                                                                 18
M4M          East Toronto      Studio District                                                                                                51
M4N          Central Toronto   Lawrence Park                                                                                                   2
M4P          Central Toronto   Davisville North                                                                                          

In [37]:
print("Number of Unique Venue Categories are: ",len(toronto_recommends['Venue_Category'].unique()))
toronto_recommends['Venue_Category'].unique()

Number of Unique Venue Categories are:  232


array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Ice Cream Shop', 'Business Service', 'Discount Store',
       'Intersection', 'Park', 'Bus Line', 'Coffee Shop',
       'Cosmetics Shop', 'Grocery Store', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Italian Restaurant',
       'Pet Store', 'Steakhouse', 'Burrito Place', 'Liquor Store',
       'Restaurant', 'Movie Theater', 'Sandwich Place', 'Pizza Place',
       'Bakery', 'Latin American Restaurant', 'Boutique',
       'Vietnamese Restaurant', 'Brewery', 'Bar', 'Diner',
       'American Restaurant', 'Antique Shop', 'Arts & Crafts Store',
       'French Restaurant', 'Gay Bar', 'Flea Market', 'Thai Restaurant',
       'Comfort Food Restaurant', 'Seafood Restaurant', 'Café',
       'Gastropub', 'Clothing Store', 'Convenience Store', 'Flower Shop',
       'Moving Target', 'Falafel Restaurant', 'Light Rail Station',
       'Furniture / Home Store', 'Swim School', 'Breakfast Spot',
       

### Analyze venues in each area

In [38]:
# getting all the values as columns for all areas
toronto_recommends = toronto_recommends.drop(['Borough_Lat','Borough_Long','Venue_Lat','Venue_Long'],axis=1)
toronto_recommends_df = pd.get_dummies(toronto_recommends, columns=['Venue_Category'],prefix = "", prefix_sep = "")
toronto_recommends_df.head(40)

,Postal Code,Borough,Neighborhoods,Venue_Name,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,Glen Manor Ravine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,The Big Carrot Natural Food Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [40]:
toronto_venues_freq = toronto_recommends_df.groupby(['Postal Code','Borough','Neighborhoods']).mean().reset_index()
toronto_venues_freq.head(40)

,Postal Code,Borough,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000

### Top 10 Venues in the each of the Areas

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postal Code', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# Create a new dataframe
all_toronto_venues = pd.DataFrame(columns=columns)
all_toronto_venues['Postal Code'] = toronto_venues_freq['Postal Code']
all_toronto_venues['Borough'] = toronto_venues_freq['Borough']
all_toronto_venues['Neighborhoods'] = toronto_venues_freq['Neighborhoods']
for ind in np.arange(toronto_venues_freq.shape[0]):
    row_categories = toronto_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    all_toronto_venues.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

all_toronto_venues.sort_values(freqColumns, inplace=True)
all_toronto_venues

,Postal Code,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,East Toronto,"The Danforth West, Riverdale",Bus Line,Cosmetics Shop,Coffee Shop,Park,Intersection,Discount Store,Grocery Store,Ice Cream Shop,Business Service,Fish & Chips Shop
4,M4N,Central Toronto,Lawrence Park,Bus Line,Swim School,Yoga Studio,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
25,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",Café,Bar,Coffee Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Pizza Place,Gaming Cafe,Thai Restaurant,Burger Joint
32,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Coffee Shop,Restaurant,Sandwich Place,Gift Shop,Nightclub,Supermarket,Japanese Restaurant,Furniture / Home Store
35,M6S,West Toronto,"Runnymede, Swansea",Café,Coffee Shop,Bakery,Pizza Place,Bank,Dance Studio,Shoe Store,Restaurant,Pub,Pilates Studio
24,M5S,Downtown Toronto,"University of Toronto, Harbord",Café,Coffee Shop,Bookstore,Bakery,Japanese Restaurant,Restaurant,Bar,Gym,Comfort Food Restaurant,Burrito Place
29,M6G,Downtown Toronto,Christie,Café,Grocery Store,Playground,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Candy Store,Farmers Market,Fast Food Restaurant
31,M6J,West Toronto,"Little Portugal, Trinity",Cocktail Bar,Bar,Coffee Shop,Asian Restaurant,Vietnamese Restaurant,Restaurant,Japanese Restaurant,Brewery,Southern / Soul Food Restaurant,Juice Bar
13,M5A,Downtown Toronto,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Spa,Event Space,Food Truck,Electronics Store,Bakery,Thai Restaurant
11,M4X,Downtown Toronto,"St. James Town, Cabbagetown",Coffee Shop,Café,Restaurant,Pizza Place,Bakery,Chinese Restaurant,Italian Restaurant,Pub,Market,Park


## Clustering areas

### Use KMeans algorigthm, try to cluster the toronto central areas into 5 clusters

In [54]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=5934aece39c014a52270ed759f2b8b45bc24e1f36a629ec95e4a7acc87963189
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [58]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_venues_freq_clustering = toronto_venues_freq.drop(['Postal Code', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_venues_freq_clustering)

#No data for a particular postal code so removed it
toronto_clustered_df = toronto_all_df.drop(toronto_all_df.index[toronto_all_df['Postal Code']=='M5N'])
toronto_clustered_df['Cluster'] = kmeans.labels_

toronto_clustered_df = toronto_clustered_df.join(all_toronto_venues.drop(['Borough', 'Neighborhoods'], 1).set_index('Postal Code'), on='Postal Code')
toronto_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_clustered_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512,0,Bus Line,Cosmetics Shop,Coffee Shop,Park,Intersection,Discount Store,Grocery Store,Ice Cream Shop,Business Service,Fish & Chips Shop
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.65351,-79.39722,0,Café,Bar,Coffee Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Pizza Place,Gaming Cafe,Thai Restaurant,Burger Joint
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.63941,-79.42676,0,Café,Bar,Coffee Shop,Restaurant,Sandwich Place,Gift Shop,Nightclub,Supermarket,Japanese Restaurant,Furniture / Home Store
36,M6S,West Toronto,"Runnymede, Swansea",43.64982,-79.47548,0,Café,Coffee Shop,Bakery,Pizza Place,Bank,Dance Studio,Shoe Store,Restaurant,Pub,Pilates Studio
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.66311,-79.40180,0,Café,Coffee Shop,Bookstore,Bakery,Japanese Restaurant,Restaurant,Bar,Gym,Comfort Food Restaurant,Burrito Place
30,M6G,Downtown Toronto,Christie,43.66869,-79.42071,0,Café,Grocery Store,Playground,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Candy Store,Farmers Market,Fast Food Restaurant
32,M6J,West Toronto,"Little Portugal, Trinity",43.64848,-79.41774,0,Cocktail Bar,Bar,Coffee Shop,Asian Restaurant,Vietnamese Restaurant,Restaurant,Japanese Restaurant,Brewery,Southern / Soul Food Restaurant,Juice Bar
13,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Spa,Event Space,Food Truck,Electronics Store,Bakery,Thai Restaurant
11,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66788,-79.36649,0,Coffee Shop,Café,Restaurant,Pizza Place,Bakery,Chinese Restaurant,Italian Restaurant,Pub,Market,Park
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Furniture / Home Store,Theater,Ramen Restaurant,Bookstore,Movie Theater


### Mapping Clusters

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

from matplotlib import cm
from matplotlib import colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_clustered_df['Latitude'], toronto_clustered_df['Longitude'], toronto_clustered_df['Postal Code'], toronto_clustered_df['Borough'], toronto_clustered_df['Neighborhood'], toronto_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5 Colours - Clusters plotted above